### 저장된 Yolo 불러오기

In [2]:
import torch

model = torch.hub.load('ultralytics/yolov5', 'custom', path = './yolov5-master/exp3/weights/best.pt')
def net_yolo(img):
    h, w, c = img.shape
    pred_list = model(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    pred_list = pred_list.pandas().xyxy[0]
    result_list = []
    for cnt in range(len(pred_list)):
        x1 = int(pred_list.loc[cnt]['xmin'])
        y1 = int(pred_list.loc[cnt]['ymin'])
        x2 = int(pred_list.loc[cnt]['xmax'])
        y2 = int(pred_list.loc[cnt]['ymax'])
        x1_nor = max(1, int(pred_list.loc[cnt]['xmin'])) / w
        y1_nor = max(1, int(pred_list.loc[cnt]['ymin'])) / h
        x2_nor = max(1, int(pred_list.loc[cnt]['xmax'])) / w
        y2_nor = max(1, int(pred_list.loc[cnt]['ymax'])) / h
        conf = round(float(pred_list.loc[cnt]['confidence']), 3)
        pred = {'bbox' : [x1, y1, x2, y2], 'conf' : conf, 'bbox_nor' : [x1_nor, y1_nor, x2_nor, y2_nor]}
        result_list.append(pred)
    if len(result_list) > 0:
        result_list.sort(key = lambda x:x['conf'], reverse = True)
        result = result_list[0]                    
        crop_img = img[result['bbox'][1]:result['bbox'][3], result['bbox'][0]:result['bbox'][2]]
        return result['bbox_nor'], crop_img
    else:
        return [], []

Using cache found in C:\Users\nyan/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-9-15 Python-3.8.15 torch-1.10.1 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11264MiB)

Fusing layers... 
Model summary: 322 layers, 86173414 parameters, 0 gradients
Adding AutoShape... 


### 저장된 Resnet 불러오기

In [3]:
def seed(seed = 1234):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
if torch.cuda.is_available() == True:
    device = 'cuda:0'
    print('현재 가상환경 GPU 사용 가능상태')
else:
    device = 'cpu'
    print('GPU 사용 불가능 상태')

현재 가상환경 GPU 사용 가능상태


In [4]:
from torchvision import models
import torch.nn as nn
from torchsummary import summary as Summary

seed()
model_resnet = models.resnet152(pretrained = True)
num_ftrs = model_resnet.fc.in_features
model_resnet.fc = nn.Linear(num_ftrs, 4)
model_resnet.load_state_dict(torch.load('./best_model_cnn.pth'))
net_cnn = model_resnet.to(device)
net_cnn.eval()
Summary(net_cnn.to(device), (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

### MediaPipe 모델 불러오기

In [5]:
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands =  mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.3)
def net_mediapipe(img):
    results = hands.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    if not results.multi_hand_landmarks: return []
    xy_list = []
    for one_hand in results.multi_hand_landmarks:
        for xy in one_hand.landmark:
            xy_list.append(xy.x)
            xy_list.append(xy.y)     
    return xy_list # 42개

### 저장된 LSTM 불러오기

In [6]:
import torch.nn as nn

class skeleton_LSTM(nn.Module) :
    def __init__(self):
        super(skeleton_LSTM, self).__init__()
        self.lstm1 = nn.LSTM(input_size=3186, hidden_size=4096, num_layers=1, batch_first=True)
        self.batch1 = nn.BatchNorm1d(30)
        self.lstm2 = nn.LSTM(input_size=4096, hidden_size=2048, num_layers=1, batch_first=True)
        self.batch2 = nn.BatchNorm1d(30)
        self.lstm3 = nn.LSTM(input_size=2048, hidden_size=1024, num_layers=1, batch_first=True)
        self.batch3 = nn.BatchNorm1d(30)
        self.dropout1 = nn.Dropout(0.3)
        self.lstm4 = nn.LSTM(input_size=1024, hidden_size=512, num_layers=1, batch_first=True)
        self.batch4 = nn.BatchNorm1d(30)
        self.lstm5 = nn.LSTM(input_size=512, hidden_size=128, num_layers=1, batch_first=True)
        self.batch5 = nn.BatchNorm1d(30)
        self.lstm6 = nn.LSTM(input_size=128, hidden_size=64, num_layers=1, batch_first=True)
        self.batch6 = nn.BatchNorm1d(30)
        self.dropout2 = nn.Dropout(0.3)
        self.lstm7 = nn.LSTM(input_size=64, hidden_size=32, num_layers=1, batch_first=True)
        self.batch7 = nn.BatchNorm1d(30)
        self.fc = nn.Linear(32,5)
    def forward(self, x) :
        x, _ = self.lstm1(x)
        x = self.batch1(x)
        x, _ = self.lstm2(x)
        x = self.batch2(x)
        x, _ = self.lstm3(x)
        x = self.batch3(x)
        x = self.dropout1(x)
        x, _ = self.lstm4(x)
        x = self.batch4(x)
        x, _ = self.lstm5(x)
        x = self.batch5(x)
        x, _ = self.lstm6(x)
        x = self.batch6(x)
        x = self.dropout2(x)
        x, _ = self.lstm7(x)
        x = self.batch7(x)
        x = self.fc(x[:,-1,:])
        return x
    
net_lstm = skeleton_LSTM()
net_lstm.load_state_dict(torch.load('./model_lstm_yolo_cnn_mediapipe_flatten_wholetrain.pth'))
net_lstm = net_lstm.to(device)
net_lstm.eval()

skeleton_LSTM(
  (lstm1): LSTM(3186, 4096, batch_first=True)
  (batch1): BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lstm2): LSTM(4096, 2048, batch_first=True)
  (batch2): BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lstm3): LSTM(2048, 1024, batch_first=True)
  (batch3): BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout1): Dropout(p=0.3, inplace=False)
  (lstm4): LSTM(1024, 512, batch_first=True)
  (batch4): BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lstm5): LSTM(512, 128, batch_first=True)
  (batch5): BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lstm6): LSTM(128, 64, batch_first=True)
  (batch6): BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout2): Dropout(p=0.3, inplace=False)
  (lstm7): LSTM(64, 32, batch_first=True)
  (batch7): BatchNorm1d(30, eps=1

### 레이블링 된 Test 데이터 돌려보기

In [7]:
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor, Resize, Normalize
import numpy as np
import numpy as np
import torch.nn.functional as F
import os
import cv2
import natsort
from tqdm import tqdm
from sklearn.preprocessing import minmax_scale
import gc
from torch.cuda import memory_allocated, empty_cache

seed()
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
data_transforms = transforms.Compose([ToTensor(), Resize((224,224)), Normalize(mean, std)])
class dataset_cnn(Dataset):
    def __init__(self, img, label):
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = data_transforms(img)
        self.img = [img]
        self.label = [label]
    def __getitem__(self, index):
        data = self.img[index]
        label = self.label[index]
        return data, label
    def __len__(self):
        return len(self.label)
    
class dataset_lstm(Dataset):
    def __init__(self, seq_list):
        self.X = []
        self.y = []
        for dic in seq_list :
            self.y.append(dic['key'])
            self.X.append(dic['value'])
    def __getitem__(self, index):
        data = self.X[index]
        label = self.y[index]
        return torch.Tensor(np.array(data)), torch.tensor(np.array(int(label)))
    def __len__(self):
        return len(self.X)
    
def clear_memory():
    if device != 'cpu':
        empty_cache()
    gc.collect()

def list_augmentation(input_list, goal_len):
    need_aug = goal_len - len(input_list)
    aug_tempo = need_aug / len(input_list)
    full_list, cnt = [], 0
    for _input in input_list:
        cnt += aug_tempo
        full_list.append(_input)
        while True:
            if cnt < 1: break
            full_list.append(_input)
            cnt -= 1
    while True:
        if len(full_list) == goal_len: break
        if len(full_list) > goal_len: del full_list[-1]
        else: full_list.append(full_list[-1])    
    return full_list

def final_model(video_path):
    last_mp_result = []
    for i in range(42):
        last_mp_result.append(0.0)
    logit_bbox_mp_list = []
    cap = cv2.VideoCapture(video_path)
    if cap.isOpened():
        while True:
            ret, img = cap.read()
            if ret == True:
                bbox, crop_img = net_yolo(img)
                if len(crop_img) == 0: continue
                flatten_img = cv2.resize(crop_img, (56, 56))
                flatten_img = cv2.cvtColor(flatten_img, cv2.COLOR_BGR2GRAY)
                flatten_img = flatten_img.reshape(56*56)
                flatten_img = list(minmax_scale(flatten_img, axis=0, copy=True))
                dataset = dataset_cnn(crop_img, 0)
                dataset = DataLoader(dataset)
                for data, label in dataset:
                    data = data.to(device)
                    with torch.no_grad():
                        cnn_result = net_cnn(data)
                    cnn_logits = []
                    for logit in cnn_result[0]:
                        cnn_logits.append(logit.item())
                    mp_result = net_mediapipe(img)
                    if len(mp_result) == 0: mp_result = last_mp_result + []
                    last_mp_result = mp_result + []
                    logit_bbox_mp_flatten = cnn_logits + bbox + last_mp_result + flatten_img
                    logit_bbox_mp_list.append(logit_bbox_mp_flatten)
            else: break
    logit_bbox_mp_list = list_augmentation(logit_bbox_mp_list, 30)
    dataset_dict_list = [{'key':0, 'value':logit_bbox_mp_list}]
    dataset = dataset_lstm(dataset_dict_list)
    dataset = DataLoader(dataset)
    for data, label in dataset:
        data = data.to(device)
        with torch.no_grad():
            result = net_lstm(data)
        _, out = torch.max(result, 1)
        pred_class = out.item()
    return pred_class

### 데이콘 제출

In [8]:
clear_memory()
video_path = './origin test'
id_list, label_list = ['id'], ['label']
video_list = os.listdir(video_path)
video_list = natsort.natsorted(video_list)
for video_name in tqdm(video_list):
    pred_class = final_model('{}/{}'.format(video_path, video_name))
    id_list.append(video_name.split('.')[0])
    label_list.append(pred_class)
with open('./result.csv', 'w', encoding = 'utf-8-sig') as f:
    cnt = 0
    for i, j in zip(id_list, label_list):
        if cnt == len(id_list) - 1:
            f.write('{},{}'.format(i, j))
            print('csv 저장 완료')
        else:
            f.write('{},{}\n'.format(i, j))
        cnt += 1
# 정답지 제작
answer_sheet = []
folder_list = os.listdir('./test_label')
for folder in folder_list:
    video_list = os.listdir('./test_label/{}'.format(folder))
    for video_name in video_list:
        answer_sheet.append('{},{}'.format(video_name.split('.')[0], folder))
answer_sheet = natsort.natsorted(answer_sheet)
# 예측지 불러오기
with open('./result.csv', 'r', encoding = 'utf-8-sig') as f:
    f = f.read()
result = f.split('\n')
del result[0]
# 정답지랑 예측지 비교
tp_list, fp_list, f1_list = [], [], []
for i in range(len(folder_list)):
    tp_list.append(0)
    fp_list.append(0)
for gt, pred in zip(answer_sheet, result):
    gt = gt.split(',')
    pred = pred.split(',')
    if gt[0] == pred[0]:
        if gt[1] == pred[1]: 
            tp_list[int(gt[1])] += 1
            status = 'TP'
        else: 
            fp_list[int(gt[1])] += 1
            status = 'FP <-- 틀림'
    else: print('Sort 여부 안 맞음')
    print('{} | GT: {} | Pred: {} | {}'.format(gt[0], gt[1], pred[1], status))
# f1 score 계산
for tp, fp in zip(tp_list, fp_list):
    f1_score = tp / (tp + fp)
    f1_list.append(f1_score)
macro_f1 = sum(f1_list) / len(f1_list)
print('Macro F1 Score: {}'.format(macro_f1))

100%|████████████████████████████████████████████████████████████████████████████████| 153/153 [09:00<00:00,  3.53s/it]

csv 저장 완료
TEST_000 | GT: 1 | Pred: 1 | TP
TEST_001 | GT: 3 | Pred: 3 | TP
TEST_002 | GT: 0 | Pred: 0 | TP
TEST_003 | GT: 2 | Pred: 2 | TP
TEST_004 | GT: 4 | Pred: 4 | TP
TEST_005 | GT: 2 | Pred: 2 | TP
TEST_006 | GT: 4 | Pred: 4 | TP
TEST_007 | GT: 3 | Pred: 3 | TP
TEST_008 | GT: 1 | Pred: 1 | TP
TEST_009 | GT: 2 | Pred: 2 | TP
TEST_010 | GT: 2 | Pred: 2 | TP
TEST_011 | GT: 2 | Pred: 2 | TP
TEST_012 | GT: 0 | Pred: 0 | TP
TEST_013 | GT: 4 | Pred: 4 | TP
TEST_014 | GT: 1 | Pred: 1 | TP
TEST_015 | GT: 4 | Pred: 4 | TP
TEST_016 | GT: 3 | Pred: 3 | TP
TEST_017 | GT: 1 | Pred: 1 | TP
TEST_018 | GT: 4 | Pred: 4 | TP
TEST_019 | GT: 1 | Pred: 1 | TP
TEST_020 | GT: 2 | Pred: 2 | TP
TEST_021 | GT: 2 | Pred: 2 | TP
TEST_022 | GT: 2 | Pred: 2 | TP
TEST_023 | GT: 3 | Pred: 3 | TP
TEST_024 | GT: 2 | Pred: 2 | TP
TEST_025 | GT: 3 | Pred: 3 | TP
TEST_026 | GT: 1 | Pred: 1 | TP
TEST_027 | GT: 4 | Pred: 4 | TP
TEST_028 | GT: 4 | Pred: 4 | TP
TEST_029 | GT: 1 | Pred: 1 | TP
TEST_030 | GT: 3 | Pred: 3 | T